In [1]:
%load_ext autoreload
%autoreload 2     

In [2]:
from voting_mechanism_design.agents.quorum_badgeholder import QuorumBadgeholder, QuorumBadgeholderPopulation
from voting_mechanism_design.projects.project import Project, ProjectPopulation
from voting_mechanism_design.funds_distribution.threshold_and_aggregate import ThresholdAndAggregate
from voting_mechanism_design.sim import RoundSimulation

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import itertools
from collections import defaultdict
from tqdm import tqdm
import pandas as pd
import warnings

# OP simulator configuration
from voting_mechanism_design.legacy.op_simulator import Simulation as OPSimulation
from voting_mechanism_design.legacy.op_simulator import Voter as OPVoter
from voting_mechanism_design.legacy.op_simulator import Project as OPProject


In [81]:
def create_badgeholders(
    n_badgeholders,
    total_funds,
    min_vote,
    max_vote,
    badgeholder_expertise_vec,
    badgeholder_laziness_vec
):
    badgeholders_n = []
    badgeholders_o = []
    for ii in range(n_badgeholders):
        agent = QuorumBadgeholder(
            badgeholder_id=ii,
            total_funds=total_funds,
            min_vote=min_vote,  
            max_vote=max_vote,
            laziness=badgeholder_laziness_vec[ii],
            expertise=badgeholder_expertise_vec[ii],
            coi_factor=0
        )
        agent2 = OPVoter(
            voter_id = ii,
            op_available=total_funds, 
            laziness=badgeholder_laziness_vec[ii],
            expertise=badgeholder_expertise_vec[ii],
        )
        badgeholders_n.append(agent)
        badgeholders_o.append(agent2)        
    return badgeholders_n,badgeholders_o

def create_projects(n_projects,project_impact_vec):
    projects_n = []
    projects_o = []
    for ii in range(n_projects):
        project = Project(
            project_id=ii,
            true_impact=project_impact_vec[ii],  
            owner_id=None,  
        )
        project2 = OPProject(
            project_id=ii,
            rating=project_impact_vec[ii],
            owner_id=None,  # No COI modeling here
        )
        projects_n.append(project)
        projects_o.append(project2)
    return projects_n,projects_o

def single_sim_old(badgeholders,projects,q,scoring_fn,total_funds,min_amount,n_projects):
    simulation=OPSimulation()
    simulation.initialize_round(max_funding=total_funds)
    simulation.round.add_voters(badgeholders)
    simulation.round.add_projects(projects)
    results = simulation.simulate_voting_and_scoring(
        n=total_funds,
        scoring_method=scoring_fn,
        quorum=q,
        min_amount=min_amount,
        normalize=True
    )
    return results

def single_sim_new(badgeholders,projects):
    badgeholder_pop=QuorumBadgeholderPopulation()
    project_population = ProjectPopulation()
    fund_distribution_model = ThresholdAndAggregate()

    badgeholder_pop.add_badgeholders(badgeholders)
    project_population.add_projects(projects)
    
    simulation_obj = RoundSimulation(
        badgeholder_population=badgeholder_pop,
        projects=project_population,
        funding_design=fund_distribution_model,
    )
    projects = project_population.get_projects()
    simulation_obj.run(cast_votes_kwargs={'projects': projects})
    return simulation_obj

def extract_old(results):
    project_data = results['data']
    project_funds_df = pd.DataFrame({
        'Project Id': [project['project_id'] for project in project_data],
        'Total Funds': [project['token_amount'] * project['num_votes'] for project in project_data]
    })
    return project_funds_df

def extract_new(results):
    project_population = results.projects
    project_vote_counts = defaultdict(int)
    for project in project_population.get_projects():
        for vote in project.votes:
            project_vote_counts[project.project_id] += vote.amount if vote.amount is not None else 0
    project_funds_df = pd.DataFrame({
        'Project Id': list(project_vote_counts.keys()),
        'Total Funds': list(project_vote_counts.values())
    })
    return project_funds_df

def plot_comparison_histograms(old, new):
    # Add framework labels
    old['Framework'] = 'Old'
    new['Framework'] = 'New'
    
    # Ensure both DataFrames have the same set of Project IDs
    combined_project_ids = pd.Series(list(set(old['Project Id']).union(set(new['Project Id'])))).sort_values()

    # Merge old and new DataFrames on Project Id to align them
    merged_df = pd.merge(old, new, on='Project Id', how='outer', suffixes=('_Old', '_New')).fillna(0)

    # Calculate the differences
    merged_df['Total Funds_Difference'] = merged_df['Total Funds_Old'] - merged_df['Total Funds_New']

    # Set the width of the bars
    bar_width = 0.25

    # Set the position of the bars on the x-axis
    r1 = np.arange(len(merged_df))
    r2 = [x + bar_width for x in r1]
    r3 = [x + bar_width for x in r2]

    # Create subplots
    fig, axs = plt.subplots(3, 1, figsize=(14, 18))

    # Plot old framework results
    axs[0].bar(r1, merged_df['Total Funds_Old'], color='blue', width=bar_width, edgecolor='grey', label='Old Framework')
    axs[0].set_xlabel('Project ID', fontweight='bold')
    axs[0].set_ylabel('Total Funds')
    axs[0].set_title('Total Funds Received by Each Project (Old Framework)')
    axs[0].legend()
    axs[0].set_xticks([r + bar_width for r in range(len(merged_df))])
    axs[0].set_xticklabels(merged_df['Project Id'])

    # Plot new framework results
    axs[1].bar(r2, merged_df['Total Funds_New'], color='green', width=bar_width, edgecolor='grey', label='New Framework')
    axs[1].set_xlabel('Project ID', fontweight='bold')
    axs[1].set_ylabel('Total Funds')
    axs[1].set_title('Total Funds Received by Each Project (New Framework)')
    axs[1].legend()
    axs[1].set_xticks([r + bar_width for r in range(len(merged_df))])
    axs[1].set_xticklabels(merged_df['Project Id'])

    # Plot the differences
    axs[2].bar(r3, merged_df['Total Funds_Difference'], color='red', width=bar_width, edgecolor='grey', label='Difference (Old - New)')
    axs[2].set_xlabel('Project ID', fontweight='bold')
    axs[2].set_ylabel('Difference in Total Funds')
    axs[2].set_title('Difference in Total Funds Received by Each Project (Old - New)')
    axs[2].legend()
    axs[2].set_xticks([r for r in range(len(merged_df))])
    axs[2].set_xticklabels(merged_df['Project Id'])

    # Adjust layout and show the plot
    plt.tight_layout()
    plt.show()
    
def sim_frameworks(
    n_badgeholders=10,
    total_funds=100,
    min_vote=1,
    max_vote=16,
    badgeholder_expertise_vec=np.ones(n_badgeholders),#will ahve to cchange this
    badgeholder_laziness_vec=np.zeros(n_badgeholders),# and this
    n_projects=100,
    project_impact_vec=np.ones(n_projects),# and this
    q=0,
    scoring_fn=['mean']
):
    badgeholders_n,badgeholders_o=create_badgeholders(
        n_badgeholders,
        total_funds,
        min_vote,
        max_vote,
        badgeholder_expertise_vec,
        badgeholder_laziness_vec
    )
    projects_n,projects_o=create_projects(n_projects,project_impact_vec)
    old_results=single_sim_old(badgeholders_o,projects_o,q,scoring_fn,total_funds,min_vote,n_projects)
    new_results=single_sim_new(badgeholders_n,projects_n)
    old=extract_old(old_results)
    new=extract_new(new_results)
    #plot_comparison_histograms(old, new)
    return old,new

def compare_rankings(old, new, detail):
    # Sort old and new DataFrames by Total Funds in descending order
    old_sorted = old.sort_values(by='Total Funds', ascending=False).reset_index(drop=True)
    new_sorted = new.sort_values(by='Total Funds', ascending=False).reset_index(drop=True)

    # Extract the rankings based on Project Id
    old_ranking = old_sorted['Project Id'].tolist()
    new_ranking = new_sorted['Project Id'].tolist()

    # Check if the rankings match
    rankings_match = old_ranking == new_ranking

    # Print the rankings
    if detail==True:
        print("Old Framework Ranking Project ID (Highest to Lowest):", old_ranking)
        print("New Framework Ranking Project ID (Highest to Lowest):", new_ranking)

    # Print whether the rankings match
    if rankings_match:
        print("The rankings match between the old and new frameworks.")
        return True
    else:
        print("The rankings do not match between the old and new frameworks.")
        
        # Highlight mismatches
        print("\nDifferences in Rankings:")
        rank_changes = {}
        for i, project_id in enumerate(old_ranking):
            old_position = i + 1
            new_position = new_ranking.index(project_id) + 1
            if old_position != new_position:
                #print(f"Project ID {project_id}: Old Rank = {old_position}, New Rank = {new_position}")
                rank_changes[project_id] = new_position - old_position

        # Count of changes
        total_changes = len(rank_changes)
        print(f"\nTotal number of projects with rank changes: {total_changes}")

        # Maximum rank change
        if rank_changes:
            max_change_project = max(rank_changes, key=lambda k: abs(rank_changes[k]))
            max_change = rank_changes[max_change_project]
            print(f"\nProject with the maximum rank change: Project ID {max_change_project}")
            print(f"Maximum rank change: {max_change} positions (Old Rank = {old_ranking.index(max_change_project) + 1}, New Rank = {new_ranking.index(max_change_project) + 1})")
        return False
    # Return the rankings for further use if needed
    #return rankings_match

In [26]:
#Variable Parameters
n_badgeholders=10
total_funds=100
min_vote=1
max_vote=16
badgeholder_expertise_vec=np.ones(n_badgeholders)
badgeholder_laziness_vec=np.zeros(n_badgeholders)
n_projects=100
project_impact_vec=np.ones(n_projects)
q=0
scoring_fn=['mean']

In [65]:
old,new=sim_frameworks(
    n_badgeholders=n_badgeholders,
    total_funds=total_funds,
    min_vote=min_vote,
    max_vote=max_vote,
    badgeholder_expertise_vec=badgeholder_expertise_vec,
    badgeholder_laziness_vec=badgeholder_laziness_vec,
    n_projects=n_projects,
    project_impact_vec=project_impact_vec,
    q=q,
    scoring_fn=scoring_fn
)
plot_comparison_histograms(old,new)
compare_rankings(old, new, False)
# i think i might also like an avg so i can see if they all move by one or if they are moving by a lot


The rankings match between the old and new frameworks.


#test variable inputs

In [ ]:
np.random.seed(42)

n_badgeholders_stress_test=(10,50,100,200)
total_funds_stress_test=(10,100,500)
badgeholder_expertise_vec_stress_test=[np.random.rand(n_badgeholders) for i in range(3)]
badgeholder_laziness_vec_stress_test=[np.random.rand(n_badgeholders) for i in range(3)]
project_impact_vec_stress_test=[np.random.rand(n_badgeholders) for i in range(3)]

experiment_cfgs = []
for n_badgeholders in n_badgeholders_stress_test:
    for total_funds in total_funds_stress_test:
        for badgeholder_expertise_vec in badgeholder_expertise_vec_stress_test:
            for badgeholder_laziness_vec in badgeholder_laziness_vec_stress_test:
                for project_impact_vec in project_impact_vec_stress_test:
                    experiment_cfgs.append((n_badgeholders, total_funds, badgeholder_expertise_vec, badgeholder_laziness_vec, project_impact_vec))


for experiment_cfg in tqdm(experiment_cfgs):
    n_badgeholders,total_funds,badgeholder_expertise_vec,badgeholder_laziness_vec,project_impact_vec=experiment_cfgs
    
    old,new=sim_frameworks(
        n_badgeholders=n_badgeholders,
        total_funds=total_funds,
        min_vote=min_vote,
        max_vote=max_vote,
        badgeholder_expertise_vec=badgeholder_expertise_vec,
        badgeholder_laziness_vec=badgeholder_laziness_vec,
        n_projects=n_projects,
        project_impact_vec=project_impact_vec,
        q=q,
        scoring_fn=scoring_fn
    )
    rankings_match=compare_rankings(old, new, False)
    if rankings_match==False:
        print(f"n_badgeholders,total_funds,badgeholder_expertise_vec,badgeholder_laziness_vec,project_impact_vec")

In [ ]:
np.random.seed(42)

n_badgeholders_stress_test = (10, 50, 100, 200)
total_funds_stress_test = (10, 100, 500)

# Generating pseudo-random vectors for stress tests
max_badgeholders= max(n_badgeholders_stress_test)
#badgeholder_expertise_vec_stress_test = [np.random.rand(max_badgeholders) for _ in range(3)]#change function for this temp
#badgeholder_laziness_vec_stress_test = [np.random.rand(max_badgeholders) for _ in range(3)]
#project_impact_vec_stress_test = [np.random.rand(max_badgeholders) for _ in range(3)]
badgeholder_expertise_vec_stress_test = list(np.linspace(0,1,4))
badgeholder_laziness_vec_stress_test = list(np.linspace(0,1,4))
project_impact_vec_stress_test = list(np.linspace(0,1,4))

experiment_cfgs = list(itertools.product(n_badgeholders_stress_test, total_funds_stress_test, badgeholder_expertise_vec_stress_test, badgeholder_laziness_vec_stress_test, project_impact_vec_stress_test))

for experiment_cfg in tqdm(experiment_cfgs):
    n_badgeholders, total_funds, badgeholder_expertise_val, badgeholder_laziness_val, project_impact_val = experiment_cfg
    
    badgeholder_expertise_vec= np.ones(max_badgeholders) * badgeholder_expertise_val
    badgeholder_laziness_vec= np.ones(max_badgeholders) * badgeholder_laziness_val
    project_impact_vec= np.ones(max_badgeholders) * project_impact_val
    
    old, new = sim_frameworks(
        n_badgeholders=n_badgeholders,
        total_funds=total_funds,
        min_vote=min_vote,
        max_vote=max_vote,
        badgeholder_expertise_vec=badgeholder_expertise_vec,
        badgeholder_laziness_vec=badgeholder_laziness_vec,
        n_projects=n_projects,
        project_impact_vec=project_impact_vec,
        q=q,
        scoring_fn=scoring_fn
    )
    rankings_match=compare_rankings(old, new, False)
    if rankings_match==False:
        print(f"Configuration:\n"
              f"- Number of Badgeholders: {n_badgeholders}\n"
        f"- Total Funds: {total_funds}\n"
        f"- Badgeholder Expertise Vector: {badgeholder_expertise_vec}\n" #ok so i want to change this so it print which vector list, not the whole list
        f"- Badgeholder Laziness Vector: {badgeholder_laziness_vec}\n"
        f"- Project Impact Vector: {project_impact_vec}\n")


  0%|▏                                                                                 | 2/768 [00:00<03:01,  4.23it/s]

The rankings match between the old and new frameworks.
The rankings match between the old and new frameworks.


  0%|▎                                                                                 | 3/768 [00:00<02:24,  5.29it/s]

The rankings match between the old and new frameworks.


  1%|▌                                                                                 | 5/768 [00:01<02:16,  5.60it/s]

...

The rankings match between the old and new frameworks.


 38%|█████████████████████████████▊                                                | 293/768 [16:24<1:05:34,  8.28s/it]

The rankings match between the old and new frameworks.


 38%|█████████████████████████████▊                                                | 294/768 [16:33<1:04:57,  8.22s/it]

The rankings match between the old and new frameworks.
